<a href="https://colab.research.google.com/github/wesley34/comp3414_course_material/blob/master/ch9_basic_reinforcement_learning/WrapperInGym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## You will learn how to wrap a gym : in working-process , not complete version



---
##### We need to wrap the gym because we want the training become more stable and better to converge.


1.   Sometimes you need to hardcode the action for start of the game, like fire the tennis in tennis game as "Fire in the beginning is a must".
2.   Due to markov property, 1 frame of photo cannot tell a good story to the model, we may need to stack "K" frames of photo together for the sake of getting "physical momentum data" like the trajectory of tennis or the car.
3.   Resize of photo can be done here
4.   Reset the gain/loss of the game




## You need the following knowledge before start

1.   oop
2.   numpy



In [ ]:
import gym
import numpy as np

In [ ]:
# you want to edit the color scale of the picture
# you need to extends the gym.ObseravtionWrapper for the purpose
# you also need to wrtie a overriden observation function 
class ScaledWrapper(gym.ObservationWrapper):
  def observation(self,obs):
    return obs/255.0

In [ ]:
# now if you make a gym, the observation ,a.k.a picture, will be normalized by 255
# create environment
wrapped_env = gym.make("Pong-v0")
wrapped_env = ScaledWrapper(wrapped_env)

env = gym.make("Pong-v0")

In [ ]:
# wrapped
wrapped_env.reset()
action = wrapped_env.action_space.sample()
observation,reward,is_done,_ = wrapped_env.step(action)
print(observation)


[[[0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  ...
  [0.56470588 0.28235294 0.06666667]
  [0.56470588 0.28235294 0.06666667]
  [0.56470588 0.28235294 0.06666667]]

 [[0.56470588 0.28235294 0.06666667]
  [0.56470588 0.28235294 0.06666667]
  [0.56470588 0.28235294 0.06666667]
  ...
  [0.56470588 0.28235294 0.06666667]
  [0.56470588 0.28235294 0.06666667]
  [0.56470588 0.28235294 0.06666667]]

 [[0.56470588 0.28235294 0.06666667]
  [0.56470588 0.28235294 0.06666667]
  [0.56470588 0.28235294 0.06666667]
  ...
  [0.56470588 0.28235294 0.06666667]
  [0.56470588 0.28235294 0.06666667]
  [0.56470588 0.28235294 0.06666667]]

 ...

 [[0.9254902  0.9254902  0.9254902 ]
  [0.9254902  0.9254902  0.9254902 ]
  [0.9254902  0.9254902  0.9254902 ]
  ...
  [0.9254902  0.9254902  0.9254902 ]
  [0.9254902  0.9254902  0.9254902 ]
  [0.9254902  0.9254902  0.9254902 ]]

 [[0.9254902  0.9254902  0.9254902 ]
  [0.9254902  0.9254902  0.9254902 ]


In [ ]:
# raw
env.reset()
action = env.action_space.sample()
observation,reward,is_done,_ = env.step(action)
print(observation)

[[[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [144  72  17]
  [144  72  17]
  [144  72  17]]

 [[144  72  17]
  [144  72  17]
  [144  72  17]
  ...
  [144  72  17]
  [144  72  17]
  [144  72  17]]

 [[144  72  17]
  [144  72  17]
  [144  72  17]
  ...
  [144  72  17]
  [144  72  17]
  [144  72  17]]

 ...

 [[236 236 236]
  [236 236 236]
  [236 236 236]
  ...
  [236 236 236]
  [236 236 236]
  [236 236 236]]

 [[236 236 236]
  [236 236 236]
  [236 236 236]
  ...
  [236 236 236]
  [236 236 236]
  [236 236 236]]

 [[236 236 236]
  [236 236 236]
  [236 236 236]
  ...
  [236 236 236]
  [236 236 236]
  [236 236 236]]]


### Example

#### stack the picture together 

In [ ]:
import cv2
import numpy as np

In [ ]:
# super initializes takes env
# 
class ProcessFrameSize84(gym.ObservationWrapper):
  def __init__(self,env=None):
    super(ProcessFrameSize84,self).__init__(env)
    self.observation_space = gym.spaces.Box(low=0,high=255,shape=(84,84,1),dtype=np.uint8)

  # override method
  # as the outside => obs,reward,game_end,_ = env.step(action) => obs will get the return from observation
  # env.step(action) will call the super observation function
  def observation(self,obs):
    return ProcessFrameSize84.process(obs)
  
  # static method 
  @staticmethod
  def process(frame):
    if frame.size == 210 * 160 * 3:
      image = np.reshape(frame,[210,160,3]).astype(np.float32)
    elif frame.size == 250 * 160 * 3:
      image = np.reshape(frame,[250,160,3]).astype(np.float32)
    else:
      assert False, "Unknown Resolution"
    
    # recall biology or physics, green color is more sensitive
    # so the weighting is higher
    # and the weighting for RGB , the sum of them equals 1
    image = image[:,:,0] * .299 + image[:,:,1] * .587 + image[:,:,2] * .114
    resized_screen = cv2.resize(image,(84,110),interpolation=cv2.INTER_AREA)
    x_t = resized_screen[18:102,:] # cut head and tail
    x_t = np.reshape(x_t,[84,84,1])
    return x_t.astype(np.uint)




In [ ]:
wrapped_env = gym.make("Pong-v0")
wrapped_env = ProcessFrameSize84(wrapped_env)
wrapped_env = ScaledWrapper(wrapped_env)

In [ ]:
# wrapped
wrapped_env.reset()
action = wrapped_env.action_space.sample()
observation,reward,is_done,_ = wrapped_env.step(action)
print(len(observation))

84


In [ ]:
wrapped_env.reset()
action = wrapped_env.action_space.sample()
observation,reward,is_done,_ = wrapped_env.step(action)
print(len(observation))

## For more
https://github.com/openai/baselines/blob/master/baselines/common/atari_wrappers.py